In [ ]:
!pip install sentencepiece
!pip install transformers
!pip install einops
!pip install rouge


In [ ]:
from transformers import pipeline,AutoTokenizer,AutoModelForSeq2SeqLM,AutoModelForCausalLM,BartTokenizer
import pandas as pd
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from transformers import pipeline, AutoTokenizer, BartTokenizer

# Corrected model names
flan_t5_base = "ybagoury/flan-t5-base-tldr_news"
bart_large = "facebook/bart-large-cnn"
pegasus = "google/pegasus-large"

bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
t5_tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
pegasus_tokenizer = AutoTokenizer.from_pretrained("google/pegasus-large")

bart_summ = pipeline("summarization", model=bart_large, tokenizer=bart_tokenizer, max_length=100, min_length=50)
t5_summ = pipeline("summarization", model=flan_t5_base, tokenizer=t5_tokenizer, max_length=100, min_length=50)
pegasus_summ = pipeline("summarization", model=pegasus, tokenizer=pegasus_tokenizer, max_length=100, min_length=50)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def getSummary(text, summarizer_model):
  # input_ids = tokenizer.encode(text, return_tensors="pt")

  # # Generate Summary Text Ids
  # summary_text_ids = model_summarize.generate(
  #     input_ids=input_ids,
  #     bos_token_id=model_summarize.config.bos_token_id,
  #     eos_token_id=model_summarize.config.eos_token_id,
  #     length_penalty=2.0,
  #     max_length=142,
  #     min_length=30,
  #     num_beams=4,
  # )

  # text_summarized=summary_text_ids[0]
  # print(tokenizer.decode(text_summarized, skip_special_tokens=True))
  text_summarized = summarizer_model(text)[0]['summary_text']

  print(len(text_summarized.split()))
  print(text_summarized)
  print("----------------------")
  return text_summarized

In [ ]:
text = '''When most presidents in the past have sent a message leading into the new year, they ve sent messages of hope, unity, and a better tomorrow. Clearly, President-elect Donald Trump is not trying to be like any former leader.Trump s New Year s Eve message, sent via his medium of choice   Twitter   went something like this: Happy New Year to all, including to my many enemies and those who have fought me and lost so badly they just don t know what to do. Love! Happy New Year to all, including to my many enemies and those who have fought me and lost so badly they just don't know what to do. Love!  Donald J. Trump (@realDonaldTrump) December 31, 2016This narcissistic message was clearly not about the nation or bringing us to together in any sort of way, but instead throws over half the nation under the bus.However, if one were to redact most of Trump s tweets and just leave the intro and the close, it would sound something like this: Happy New Year to all. Love! Redacted, Trump sounds just like Obama! ? pic.twitter.com/h9W5JbqE98  Sarah Wood (@SarahWoodwriter) December 31, 2016Which, quite honestly, sounds like someone who wants to unite the nation and move forward towards a better tomorrow. It actually sounds remarkably like the president we ve had for the past eight years, President Barack Obama.If only Trump could contain his narcissistic animosity towards those who don t like him or agree with him. However, his fragile ego doesn t seem to make that possible thus his constant Twitter fits of rage against anyone who dare speak against him.Redacted Trump could actually be a good leader. Unfortunately, he doesn t seem to know how to edit himself for the greater good. Hopefully that will change.Featured Photo by Chip Somodevilla, Twitter/Getty Images
'''
reference_summary = '''President-elect Donald Trump's New Year's Eve message on Twitter was criticized for being narcissistic and not promoting unity or hope. If redacted, Trump's message would sound more like President Barack Obama, aiming to unite the nation and move towards a better tomorrow. However, Trump's fragile ego makes it difficult for him to edit himself for the greater good.
'''

In [ ]:
print("Flan T5 summarization: \n")
t5_summarized = t5_summ(text)[0]['summary_text']
print(t5_summarized)

Flan T5 summarization: 

Donald Trump’s New Year s Eve message is a narcissistic one, and it’s not about the nation or bringing us together in any sort of way, but instead throws over half the nation under the bus


In [ ]:
print("Bart summarization: \n")
bart_summarized = bart_summ(text)[0]['summary_text']
print(bart_summarized)

Bart summarization: 

Trump's New Year s Eve message was clearly not about the nation or bringing us to together in any sort of way, but instead throws over half the nation under the bus. If one were to redact most of Trump s tweets and just leave the intro and the close, it would sound something like this: Happy New Year to all. Love!


In [ ]:
print("Pegasus summarization: \n")
pegasus_summarized = pegasus_summ(text)[0]['summary_text']
print(pegasus_summarized)

Pegasus summarization: 

Clearly, President-elect Donald Trump is not trying to be like any former leader.Trump s New Year s Eve message, sent via his medium of choice Twitter went something like this: Happy New Year to all, including to my many enemies and those who have fought me and lost so badly they just don t know what to do. Trump (@realDonaldTrump) December 31, 2016This narcissistic message was clearly not about the nation or bringing us to together in any sort of way


In [ ]:
from rouge import Rouge

model_summary = t5_summarized

# Create a Rouge object
rouge = Rouge()

# Compute ROUGE scores
scores = rouge.get_scores(model_summary, reference_summary)

# Print ROUGE scores
print(scores)


[{'rouge-1': {'r': 0.21568627450980393, 'p': 0.3333333333333333, 'f': 0.26190475713435374}, 'rouge-2': {'r': 0.034482758620689655, 'p': 0.058823529411764705, 'f': 0.04347825620983037}, 'rouge-l': {'r': 0.19607843137254902, 'p': 0.30303030303030304, 'f': 0.23809523332483}}]


In [ ]:
model_summary = bart_summarized

# Create a Rouge object
rouge = Rouge()

# Compute ROUGE scores
scores = rouge.get_scores(model_summary, reference_summary)

# Print ROUGE scores
print(scores)


[{'rouge-1': {'r': 0.3137254901960784, 'p': 0.32, 'f': 0.316831678168807}, 'rouge-2': {'r': 0.06896551724137931, 'p': 0.06779661016949153, 'f': 0.068376063376434}, 'rouge-l': {'r': 0.29411764705882354, 'p': 0.3, 'f': 0.29702969797078727}}]


In [ ]:
model_summary = pegasus_summarized

# Create a Rouge object
rouge = Rouge()

# Compute ROUGE scores
scores = rouge.get_scores(model_summary, reference_summary)

# Print ROUGE scores
print(scores)


[{'rouge-1': {'r': 0.29411764705882354, 'p': 0.22058823529411764, 'f': 0.25210083543817535}, 'rouge-2': {'r': 0.034482758620689655, 'p': 0.025, 'f': 0.028985502373451773}, 'rouge-l': {'r': 0.27450980392156865, 'p': 0.20588235294117646, 'f': 0.23529411274909975}}]
